In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
import torchvision
import torchvision.datasets as data
import torchvision.transforms as trans
from torch.utils.data.dataset import Dataset
from torch.utils.tensorboard import SummaryWriter as summary
from torch.autograd import backward
import matplotlib.pyplot as plt
import torchbearer
from torchbearer import Trial
from torchbearer.cv_utils import DatasetValidationSplitter

In [ ]:
# CREATE DIRECTORIES USED IN THIS PROGRAM HERE

if not os.path.exists("data/MNIST"):
    os.makedirs("data/MNIST")

if not os.path.exists("data/VAE/saved"):
    os.makedirs("data/VAE/saved")

if not os.path.exists("data/VAE/logs"):
    os.makedirs("data/VAE/logs")
    
# Directory configuration
dataset_dir  = "data/MNIST"
output_dir   = "data/VAE/"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class AutoEncoderMNIST(Dataset):
    def __init__(self, mnist_dataset):
        super().__init__()
        self.mnist_dataset = mnist_dataset

    def __getitem__(self, index):
        character, label = self.mnist_dataset.__getitem__(index)
        return character, character

    def __len__(self):
        return len(self.mnist_dataset)
    
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


def bce_loss(recon_x, x):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    return BCE


def kld_Loss(mu, logvar):
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return KLD


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(y_pred, y_true):
    recon_x, mu, logvar = y_pred
    x = y_true

    BCE = bce_loss(recon_x, x)

    KLD = kld_Loss(mu, logvar)

    return BCE + KLD


In [ ]:
batch_size=32

normalize = trans.Compose([trans.ToTensor()])

dataset = torchvision.datasets.MNIST('./data/mnist', train=True, download=True, transform=normalize)
splitter = DatasetValidationSplitter(len(dataset), 0.1)
base_train_set = splitter.get_train_dataset(dataset)
base_val_set = splitter.get_val_dataset(dataset)

train_set = AutoEncoderMNIST(base_train_set)

val_set = AutoEncoderMNIST(base_val_set)

train_gen = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)

val_gen = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=8)

In [ ]:
def add_kld_loss_callback(state):
    KLD = kld(state[MU], state[LOGVAR])
    return KLD

def save_reconstruction_callback(num_images=8, folder='results/'):
    import os
    os.makedirs(os.path.dirname(folder), exist_ok=True)

    @torchbearer.callbacks.on_step_validation
    def saver(state):
        if state[torchbearer.BATCH] == 0:
            data = state[torchbearer.X]
            recon_batch = state[torchbearer.Y_PRED]
            comparison = torch.cat([data[:num_images],
                                    recon_batch.view(128, 1, 28, 28)[:num_images]])
            save_image(comparison.cpu(),
                       str(folder) + 'reconstruction_' + str(state[torchbearer.EPOCH]) + '.png', nrow=num_images)
    return saver

In [4]:
model = VAE()
optimizer = opt.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
loss = loss_function
tb_run = Trial(model, optimizer, loss, metrics=['loss']).to('cuda')
tb_run.with_generators(train_generator=traingen, val_generator=valgen)
tb_run.run(epochs=1)

NameError: name 'VAE' is not defined

In [5]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir="C:/Users/Nick/results"

Launching TensorBoard...

NameError: name 'ipynb' is not defined